# Imports

In [1]:
import asyncio
from uuid import uuid4

In [2]:
from pydantic import BaseModel

In [3]:
from matter_persistence.redis.manager import CacheManager
from matter_persistence.foundation_model import FoundationModel
from matter_persistence.redis.utils import get_connection_pool

# UUIDS

In [4]:
# random organisation and internal id
ORGANISATION_ID = uuid4()
INTERNAL_ID = uuid4()

# Example Pydantic type

In [5]:
# example pydantic type
class Person(FoundationModel):
    name: str
    age: int

# test type
john = Person(name="John", age=30)

In [6]:
john # created_at & created_at_timestamp are added "automatically"

Person(created_at=datetime.datetime(2024, 5, 16, 13, 24, 6, 291991, tzinfo=datetime.timezone.utc), created_at_timestamp=1715865846.292016, name='John', age=30)

# Create Person from different objects

## from python dict

In [7]:
Person.parse_obj({'name': 'Jane', 'age': 30})

Person(created_at=datetime.datetime(2024, 5, 16, 13, 24, 6, 291991, tzinfo=datetime.timezone.utc), created_at_timestamp=1715865846.292016, name='Jane', age=30)

## from other pydantic type

In [8]:
class PersonWithOnlyName(BaseModel):
    name: str

class PersonWithHairColour(BaseModel):
    name: str
    age: int
    hair_colour: str

In [9]:
# the required fields must be present in the other pydantic type!!

In [10]:
Person.parse_obj(PersonWithHairColour(name="Simone", age=30, hair_colour="brown"))  # extra fields are ignored

Person(created_at=datetime.datetime(2024, 5, 16, 13, 24, 6, 291991, tzinfo=datetime.timezone.utc), created_at_timestamp=1715865846.292016, name='Simone', age=30)

# CacheManager

In [11]:
connection_pool = get_connection_pool(host="localhost", port=6379)
cache_manager = CacheManager(connection_pool=connection_pool) # make sure there is a redis instance running!

# Save, get, and delete pydantic type

In [12]:
# check if cache is alive
await cache_manager.is_cache_alive()

True

In [13]:
# save pydantic type to cache
await cache_manager.save_value(ORGANISATION_ID, INTERNAL_ID, john, Person)

In [14]:
# check if pydantic object exists in cache
await cache_manager.cache_record_exists(ORGANISATION_ID, INTERNAL_ID, Person)

True

In [15]:
# find pydantic object in cache
await cache_manager.get_value(ORGANISATION_ID, INTERNAL_ID, Person)

CacheRecordModel(created_at=datetime.datetime(2024, 5, 16, 13, 24, 6, 291991, tzinfo=datetime.timezone.utc), created_at_timestamp=1715865846.292016, internal_id='9fa9ba40-a660-4d0b-b985-c9e341549322', hash_key='0e6745f0-0951-4568-b982-e455855090f5_Person_4938eaa990a08f446a0506af24ba4628ea2f3850', organization_id=UUID('0e6745f0-0951-4568-b982-e455855090f5'), value=Person(created_at=datetime.datetime(2024, 5, 16, 13, 24, 6, 291991, tzinfo=datetime.timezone.utc), created_at_timestamp=1715865846.292016, name='John', age=30), expiration=None)

In [16]:
# delete pydantic object from cache
await cache_manager.delete_value(ORGANISATION_ID, INTERNAL_ID, Person)

In [17]:
# make sure pydantic object is deleted from cache
await cache_manager.cache_record_exists(ORGANISATION_ID, INTERNAL_ID, Person)

False

# Save, get, and delete with key

In [18]:
await cache_manager.save_with_key("key", "value")

True

In [19]:
await cache_manager.save_with_key("john", john, Person)

True

In [20]:
await cache_manager.get_with_key("key")

b'value'

In [21]:
await cache_manager.get_with_key("john", Person)

Person(created_at=datetime.datetime(2024, 5, 16, 13, 24, 6, 291991, tzinfo=TzInfo(UTC)), created_at_timestamp=1715865846.292016, name='John', age=30)

In [22]:
await cache_manager.cache_record_with_key_exists("key")

True

In [26]:
await cache_manager.cache_record_with_key_exists("john", Person)

True

In [24]:
await cache_manager.delete_with_key("key")

In [27]:
await cache_manager.delete_with_key("john", Person)

# Close connection pool

In [28]:
await cache_manager.close_connection_pool()